### Fairy Tales using Gemini AI

The idea of this code is to make it simpler to make questions about some fairy tales (they need to be saved under `fairy_tales` folder), to summarize them and make the language easier to understand, for example.
You can even ask the question in another language, such as portuguese.

#### Get Key and pass it to genai 

In [ ]:
import os
import numpy as np
import pandas as pd
from dotenv import load_dotenv, dotenv_values 
import google.generativeai as genai

load_dotenv() 
api_key = os.getenv("API_SECRET_KEY")
genai.configure(api_key=api_key)

#### Get the documents

In [2]:
file_names = os.listdir("fairy_tales")
tales = {}
documents = []
for file in file_names:
    with open(f"fairy_tales/{file}", "r") as fname:
        tale_name = file.split(".")[0]
        content = fname.readlines()
    documents.append(
        {
            "title": tale_name,
            "text": "".join(content)
        }
    )

In [ ]:
# Check the documents
documents

#### Generate the Dataframe

In [4]:
df = pd.DataFrame(documents)
#df = df.set_index("title")["text"].str.split("\n", expand=True).stack()
df

,title,text
0,drop_of_water,Of course you know what is meant by a magnifyi...
1,butterfly,There was once a butterfly who wished for a br...
2,loveliest_rose_in_the_world,"There lived once a great queen, in whose garde..."


#### Generate the embeddings

In [5]:
model = "models/embedding-001"
task_type = "RETRIEVAL_DOCUMENT"
def gen_embed_content(title, text):
  return genai.embed_content(model=model,
                            content=text,
                            title=title,
                            task_type=task_type)["embedding"]

In [6]:
df["embeddings"] = df.apply(lambda row: gen_embed_content(row["title"], row["text"]), 
                            axis=1)

In [7]:
df

,title,text,embeddings
0,drop_of_water,Of course you know what is meant by a magnifyi...,"[0.0030424898, -0.009695117, -0.009657833, -0...."
1,butterfly,There was once a butterfly who wished for a br...,"[0.008295853, -0.055345494, -0.04492399, -0.03..."
2,loveliest_rose_in_the_world,"There lived once a great queen, in whose garde...","[0.027745383, 0.0076825176, -0.053658728, -0.0..."


#### Generate query embedding and search in the docs

In [8]:
def gen_embed_and_search(model, dataframe, query):
    query_embedding = genai.embed_content(model=model,
                                 content=query,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

    dot_products = np.dot(np.stack(df["embeddings"]), query_embedding)

    return df.iloc[np.argmax(dot_products)]["text"]

#### Config the Generative Model to generate content

In [9]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [10]:
gemini_pro_model = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)

### Make your queries

In [11]:
print("Choose the final word for your queries")
final_word = input()

Choose the final word for your queries


 end


In [15]:
query = ""
prompt = "Get the text and improve the answer to the question: {question}:{answer}. Your response should be based only on the text given on the answer. You can adapt the language and make it simpler. The final response has to follow the same language used in the question."
while query != final_word:
    print("======================================================")
    query = input(">>Waiting for your query about Fairy Tails: ")
    if query == final_word:
        continue
    answer = gen_embed_and_search(model, df, query)
    prepared_prompt = prompt.format(
        question = query,
        answer = answer
    )
    response = gemini_pro_model.generate_content(prepared_prompt)
    print(response.text)

>>Waiting for your query about Fairy Tails:  What is the source of the texts?


Source of the text: Project Gutenberg


>>Waiting for your query about Fairy Tails:  O que a borboleta queria?


O que a borboleta queria?: A borboleta queria uma noiva bonita entre as flores.


>>Waiting for your query about Fairy Tails:  Summarize the drop of water story for a 5-year-old child.


Once upon a time, there was a man named Kribble-Krabble who loved to look at things through a special glass that made them look bigger. One day, he looked at a drop of water from a puddle and saw lots of tiny creatures swimming around.

But these creatures were not friendly! They were fighting and biting each other. Kribble-Krabble thought it was terrible and wanted them to be peaceful.

So, he used magic to make the creatures pink so he could see them better. He showed them to another magician, who didn't know what they were.

The other magician looked through the glass and saw a big city with people running around without clothes. He was shocked to see how they were fighting and hurting each other.

"That's a drop of puddle water!" Kribble-Krabble said. "It shows us how people in big cities can be mean and unkind."


>>Waiting for your query about Fairy Tails:  What could save the queen?


**What could save the queen?**

The queen was very sick, and the doctors said she would die. But one of the wisest doctors said there was one thing that could save her: the loveliest rose in the world, one that showed the purest and brightest love.

Many people brought roses from all over the world, but none of them were the right one. The rose had to be from the garden of love, and it had to show the highest and purest love.

Finally, a little child came into the room and read from a book about the love of Jesus Christ. As he read, a roseate hue spread over the queen's cheeks, and her eyes became so enlightened and clear, that she saw from the leaves of the book a lovely rose spring forth, a type of Him who shed His blood on the cross.

"I see it," she said. "He who beholds this, the loveliest rose on earth, shall never die."


>>Waiting for your query about Fairy Tails:  end
